# Sesión 2: Lectura de Datos y Análisis con Pandas en Ciencias Ambientales

## Parte 1: Lectura de Archivos y Consumo de APIs

### 1.1 Lectura de Archivos CSV


In [7]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Leer un archivo CSV con datos de calidad del agua
df_water_quality = pd.read_csv('https://raw.githubusercontent.com/gonzalezulises/101_Data_Analytics_Rizoma/master/data/water_potability.csv')
print(df_water_quality.head())

In [ ]:
### 1.2 Lectura de Archivos Excel

# Leer un archivo Excel con datos meteorológicos
df_weather = pd.read_excel('/Users/gonzalezulises/Documents/GitHub/101_Data_Analytics_Rizoma/data/Pronostico_del_tiempo.xlsx', sheet_name='Hoja 1 - data (1)')
print(df_weather.head())

In [ ]:
### 1.3 Consumo de API

import requests

# Obtener datos de calidad del aire de una API
api_url = "https://api.openaq.org/v1/latest?country=ES&parameter=pm25"
response = requests.get(api_url)
air_quality_data = response.json()

# Convertir los datos de la API a un DataFrame
df_air_quality = pd.DataFrame(air_quality_data['results'])
print(df_air_quality.head())


In [ ]:
## Parte 2: Análisis de Datos con Pandas

### 2.1 Exploración Inicial de Datos

# Usando el DataFrame de calidad del agua
print(df_water_quality.info())
print(df_water_quality.describe())

# Verificar valores nulos
print(df_water_quality.isnull().sum())


In [ ]:

### 2.2 Selección y Filtrado de Datos

# Seleccionar columnas específicas
ph_and_temp = df_water_quality[['pH', 'Temperature']]

# Filtrar datos
high_ph = df_water_quality[df_water_quality['pH'] > 8]

In [ ]:
### 2.3 Agrupación y Agregación

# Agrupar por ubicación y calcular promedios
avg_by_location = df_water_quality.groupby('Location').mean()
print(avg_by_location)

In [ ]:
## 2.4 Operaciones con Series y DataFrames

# Añadir una nueva columna
df_water_quality['pH_category'] = pd.cut(df_water_quality['pH'], 
                                         bins=[0, 6.5, 7.5, 14],
                                         labels=['Acidic', 'Neutral', 'Alkaline'])

# Operaciones entre columnas
df_weather['temp_difference'] = df_weather['max_temp'] - df_weather['min_temp']

In [ ]:
### 2.5 Manejo de Datos Faltantes

# Rellenar valores nulos
df_water_quality['Dissolved_Oxygen'].fillna(df_water_quality['Dissolved_Oxygen'].mean(), inplace=True)

# Eliminar filas con valores nulos
df_water_quality.dropna(subset=['pH'], inplace=True)

In [ ]:
### 2.6 Visualización Básica con Pandas

# Gráfico de líneas para temperatura a lo largo del tiempo
df_weather.plot(x='date', y='avg_temp', kind='line')

# Histograma de pH
df_water_quality['pH'].hist(bins=20)



## Ejercicios Prácticos

- Carga un conjunto de datos de precipitaciones mensuales desde el archivo CSV (https://raw.githubusercontent.com/gonzalezulises/detodounpoco/main/PREC_2021_Provincias.csv) y resuelve los siguientes ejercicios:

1. Calcula la precipitación anual por región y muestra las 5 regiones más lluviosas. Usa `groupby()` para agrupar por región, `sum()` para sumar precipitaciones, `sort_values()` para ordenar y `head()` para mostrar las top 5.

2. Encuentra los 3 meses más lluviosos y los 3 más secos en promedio para toda España. Utiliza `melt()` para reestructurar el DataFrame, `groupby()` para agrupar por mes, `mean()` para calcular promedios, `sort_values()` para ordenar y `head()` para seleccionar los extremos.

3. Agrupa las regiones por niveles de precipitación anual (bajo: <500mm, medio: 500-1000mm, alto: >1000mm). Emplea `cut()` para categorizar las precipitaciones y `value_counts()` para contar las regiones en cada categoría.

4. Crea un DataFrame con la variación mensual de precipitaciones para las 5 regiones más lluviosas. Usa `set_index()` para establecer las regiones como índice, `nlargest()` para seleccionar las top 5 regiones basadas en precipitación anual, y `T` para transponer el resultado si es necesario.

5. Identifica regiones con condiciones extremas: a) al menos un mes >200mm, b) al menos tres meses consecutivos <10mm. Utiliza `melt()` para reestructurar, `groupby()` con `any()` para identificar meses >200mm, y `rolling()` con `all()` para periodos consecutivos <10mm.

6. Calcula la correlación entre la precipitación de verano (junio, julio, agosto) y la anual. Selecciona las columnas relevantes con `[]` y usa `corr()` para calcular la correlación.

7. Prepara un DataFrame para un mapa de calor de precipitaciones mensuales por región. Emplea `set_index()` para establecer las regiones como índice y selecciona solo las columnas de los meses con `[]`.

8. Analiza la estacionalidad calculando el porcentaje de precipitación por estación para cada región. Agrupa los meses por estaciones usando `[]`, luego aplica `groupby()` y `sum()` para totales estacionales, y `apply()` con una función lambda para calcular porcentajes.

9. Compara precipitaciones entre regiones costeras e interiores. Usa `isin()` para clasificar regiones como costeras o interiores, `groupby()` para agrupar por tipo, y `agg()` para calcular estadísticas descriptivas por grupo.

10. Identifica posibles anomalías en los datos de precipitación mensual. Utiliza `melt()` para reestructurar, `groupby()` con `quantile()` para calcular umbrales de anomalías, y `query()` para filtrar valores atípicos basados en estos umbrales.



## Soluciones

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/gonzalezulises/detodounpoco/main/PREC_2021_Provincias.csv')

In [ ]:
# 1. Precipitación anual por región y top 5 más lluviosas
precipitacion_anual = df.groupby('region')['anual'].sum().sort_values(ascending=False)
print("Top 5 regiones más lluviosas:")
print(precipitacion_anual.head())


In [ ]:

# 2. 3 meses más lluviosos y 3 más secos en promedio
df_melt = df.melt(id_vars=['Parametro', 'region', 'anual'], 
                  var_name='mes', value_name='precipitacion')
precipitacion_mensual = df_melt.groupby('mes')['precipitacion'].mean().sort_values(ascending=False)
print("\n3 meses más lluviosos:")
print(precipitacion_mensual.head(3))
print("\n3 meses más secos:")
print(precipitacion_mensual.tail(3))


In [ ]:

# 3. Agrupar regiones por niveles de precipitación anual
df['categoria_precipitacion'] = pd.cut(df['anual'], 
                                       bins=[0, 500, 1000, float('inf')], 
                                       labels=['Bajo', 'Medio', 'Alto'])
conteo_categorias = df['categoria_precipitacion'].value_counts()
print("\nConteo de regiones por categoría de precipitación:")
print(conteo_categorias)


In [ ]:

# 4. Variación mensual de precipitaciones para las 5 regiones más lluviosas
top_5_regiones = df.nlargest(5, 'anual')['region'].tolist()
df_top_5 = df[df['region'].isin(top_5_regiones)].set_index('region')
df_top_5_mensual = df_top_5.loc[:, 'enero':'diciembre']
print("\nVariación mensual de precipitaciones para las 5 regiones más lluviosas:")
print(df_top_5_mensual)


In [ ]:

# 5. Identificar regiones con condiciones extremas
df_melt = df.melt(id_vars=['Parametro', 'region', 'anual'], 
                  var_name='mes', value_name='precipitacion')
regiones_extremas = df_melt.groupby('region').agg(
    max_precipitacion=('precipitacion', 'max'),
    meses_secos=('precipitacion', lambda x: (x < 10).rolling(3).sum().max())
)
regiones_extremas = regiones_extremas[
    (regiones_extremas['max_precipitacion'] > 200) | 
    (regiones_extremas['meses_secos'] >= 3)
]
print("\nRegiones con condiciones extremas:")
print(regiones_extremas)


In [ ]:

# 6. Correlación entre precipitación de verano y anual
df['precipitacion_verano'] = df[['junio', 'julio', 'agosto']].sum(axis=1)
correlacion = df['precipitacion_verano'].corr(df['anual'])
print(f"\nCorrelación entre precipitación de verano y anual: {correlacion:.2f}")


In [ ]:

# 7. Preparar DataFrame para mapa de calor
df_heatmap = df.set_index('region').loc[:, 'enero':'diciembre']
print("\nDataFrame para mapa de calor de precipitaciones mensuales:")
print(df_heatmap.head())


In [ ]:

# 8. Analizar estacionalidad
df['primavera'] = df[['marzo', 'abril', 'mayo']].sum(axis=1)
df['verano'] = df[['junio', 'julio', 'agosto']].sum(axis=1)
df['otono'] = df[['septiembre', 'octubre', 'noviembre']].sum(axis=1)
df['invierno'] = df[['diciembre', 'enero', 'febrero']].sum(axis=1)

df_estaciones = df[['region', 'primavera', 'verano', 'otono', 'invierno', 'anual']]
df_estaciones_pct = df_estaciones.set_index('region').apply(lambda x: x / x['anual'] * 100)
print("\nPorcentaje de precipitación por estación:")
print(df_estaciones_pct.head())


In [ ]:

# 9. Comparar precipitaciones entre regiones costeras e interiores
regiones_costeras = ['A CORUNA', 'ASTURIAS', 'CANTABRIA', 'BIZKAIA', 'GIPUZKOA', 'BARCELONA', 
                     'TARRAGONA', 'CASTELLON', 'VALENCIA', 'ALICANTE', 'MURCIA', 'ALMERIA', 
                     'GRANADA', 'MALAGA', 'CADIZ', 'HUELVA', 'PONTEVEDRA', 'ILLES BALEARS', 
                     'LAS PALMAS', 'SANTA CRUZ DE TENERIFE', 'CEUTA', 'MELILLA']

df['tipo_region'] = df['region'].apply(lambda x: 'Costera' if x in regiones_costeras else 'Interior')
comparacion = df.groupby('tipo_region')['anual'].agg(['mean', 'median', 'std'])
print("\nComparación de precipitaciones entre regiones costeras e interiores:")
print(comparacion)


In [ ]:

# 10. Identificar posibles anomalías
df_melt = df.melt(id_vars=['Parametro', 'region', 'anual'], 
                  var_name='mes', value_name='precipitacion')
df_melt['z_score'] = df_melt.groupby('mes')['precipitacion'].transform(
    lambda x: (x - x.mean()) / x.std()
)
anomalias = df_melt[df_melt['z_score'].abs() > 3]
print("\nPosibles anomalías en los datos de precipitación:")
print(anomalias[['region', 'mes', 'precipitacion', 'z_score']])